In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.ndimage.filters import maximum_filter
import timeit
import cv2
import mysql.connector
import sys
import os
import hashlib
import sounddevice as sd
import librosa
import pyaudio
import wave

In [ ]:
from Parameters import FRAME_SIZE, HOP_SIZE 
def spectrogram(recarr,sr):
    
    stft_recarr=librosa.stft(recarr,n_fft=FRAME_SIZE,hop_length=HOP_SIZE)
    spec_matrix = (np.abs(stft_recarr))**2
    
    return spec_matrix

In [ ]:
from Parameters import peak_neighborhood_size, min_amp, size

def get_peaks(stft_array):
    
    local_max = maximum_filter(stft_array, size) == stft_array  #Comparing with the original array gives us the sparse array 
                                                                #in the form of a boolean matrix


    frequency_list = []
    time_list = []
    
    L = stft_array.shape[0]   # No of rows in the stft_array
    B = stft_array.shape[1]   # No of columns in the stft_array

    #The first 2 loops ar used to iterate through all the local maximas
    #The following 2 nested if statements are used to check if a particular local maxima is in the required target zone
    #Each time we find a peak, the third for loop is used which finds additional 5-6 peaks in the same vertical bin
    for i in range(B):
        for j in range(L):
            if (local_max[j][i]): 
                
                if (stft_array[j][i]>min_amp):           
                    frequency_list.append(j)
                    time_list.append(i)
                    
                else:
                    local_max[j][i] = False
                
                
                vertical_bin = np.argpartition(stft_array[:,i], -6)[-6:]
                
                for k in range(len(vertical_bin)):
                    if (stft_array[vertical_bin[k]][i]>min_amp):
                        frequency_list.append(vertical_bin[k])
                        time_list.append(i)
                        

    #Use this code for plotting the peaks                
#     plt.imshow(local_max, cmap='Greys',  interpolation='nearest')
#     plt.show()

    return time_list, frequency_list, L, B

In [ ]:
from Parameters import min_time_delta, max_time_delta, fan_value, range_frequencies

def get_hashes(time_list, frequency_list, L, B,song_id):
    t = time_list
    f = frequency_list
    
    N = len(time_list)
    hashes = []

    count2 = 0
    
    for i in range(N):
        for j in range(min_time_delta,max_time_delta):
            if ((i+j)<N):
                frequency1 = f[i]
                frequency2 = f[i+j]
                time_delta = t[i+j] - t[i]
                
                if ( (time_delta > min_time_delta) & (time_delta < max_time_delta) ):
                    count2 += 1
                    h = hashlib.sha224(f"{str(frequency1)}|{str(frequency2)}|{time_delta}".encode('utf-8'))
                    hash1 = h.hexdigest()
                    hashes.append((song_id,hash1[0:20],t[i]))
                    if (count2>=fan_value):
                        break
                        
    return hashes

In [ ]:
from mysql_config import host, user, password, database

def add_songs_to_database():
    
    #Connecting to the MySQL server
    try:
        mydb = mysql.connector.connect(host = host,
                                       user = user,
                                       password = password,
                                       database = database)
    except:
        print("Unable to conect to the database")
    
    mycursor = mydb.cursor()
    
    #Reading the song
    song_name = input("Enter song name ")
    filename='mp3/'+song_name+'.wav'
    recarr,sr=librosa.load(filename)
    
    
    #Checking if the song is already fingerprinted and if not then adding it in the database
    sql = "SELECT count(*) from songs where song_name = %s and fingerprinted = %s"
    val = (song_name,"1")
    mycursor.execute(sql,val)
    x = mycursor.fetchall()
    if (x[0][0]==1):
        print("Song ", song_name," is already fingerprinted")
        return
    else:
        sql = "INSERT INTO songs (song_name,fingerprinted) VALUES (%s,%s)"
        val = (song_name,"1")
        mycursor.execute(sql,val)
        sql = "SELECT * from songs where song_name = %s and fingerprinted = %s"
        val = (song_name,"1")
        mycursor.execute(sql,val)
        try:
            x = mycursor.fetchall()
            song_id = x[0][0]
        except:
            song_id = 1
        
    print("Making spectrogram")
    spec_matrix = spectrogram(recarr,sr)
    print("Spectrogram successfully made")
    
    print("Finding peaks in the spectrogram")
    time_list, frequency_list, L, B = get_peaks(spec_matrix)
    print("Found ", len(time_list)," peaks")
    
    print("Making hashes")
    hashes = get_hashes(time_list, frequency_list, L, B,song_id)
    print("Made ",len(hashes)," hashes")
    hashes = list(set(hashes))
    print("Found ",len(hashes)," unique hashes")
    
    #Saving hashes in the databses
    val = hashes
    sql = "INSERT INTO fingerprints (song_id,hash,offset) VALUES (%s,%s,%s)"
    mycursor.executemany(sql,val)
    mydb.commit()
    print(mycursor.rowcount, "record(s) inserted for song ",song_name)
    
    #Closing the connection with the database
    mycursor.close()
    mydb.close()
    

In [ ]:
def recognize():
    
    sampling_rate = 44100
    duration = 10
    print("How do you want to recognize the song")
    print("    1. Through microphone")
    print("    2. Reading the sample file from system")
    choice = input("Enter your choice number: ")
    
    if(choice == '1'):
        filename = "test100" + ".wav"
        chunk = 1024
        FORMAT = pyaudio.paInt16
        channels = 1
        sample_rate = 44100
        record_seconds = 10
        p = pyaudio.PyAudio()
        stream = p.open(format=FORMAT,
                channels=channels,
                rate=sample_rate,
                input=True,
                output=True,
                frames_per_buffer=chunk)
        frames = []
        print("Recording...")        
        for i in range(int(44100 / chunk * record_seconds)):
            data = stream.read(chunk)
            frames.append(data)
        
        print("Recording ended...")
        stream.stop_stream()
        stream.close()
        p.terminate()
        wf = wave.open(filename, "wb")
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(sample_rate)
        wf.writeframes(b"".join(frames))
        wf.close()
        

        path = filename 
        recarr,sr=librosa.load(path)
        recognize_the_song(recarr, sampling_rate)
    
    
    elif (choice == '2'):
        sample_name = input("Enter the name of the sample you want to recognize: ")        
        path = 'test/' + sample_name + '.wav' 
        recarr,sr=librosa.load(path)
        recognize_the_song(recarr,sr)
        
    else:
        print("PLease enter a valid choice number")
        
    return

In [ ]:
from mysql_config import host, user, password, database

def recognize_the_song(audio, sampling_rate):
    
    #Connecting to the MySQL server
    try:
        mydb = mysql.connector.connect(host = host,
                                       user = user,
                                       password = password,
                                       database = database)
    except:
        print("Unable to conect to the database")
    
    mycursor = mydb.cursor()
    
    print("Making spec")
    spec_matrix = spectrogram(audio,sampling_rate)
    print("Spec done")
    
    print("Finding peaks")
    time_list, frequency_list, L, B = get_peaks(spec_matrix)
    print("Peaks found", len(time_list))
    
    hashes = get_hashes(time_list, frequency_list, L, B,song_id = 0)
    print("Hashes found",len(hashes))
    hashes = list(set(hashes))
    print("Unique Hashes found",len(hashes))
    
    unique_hashes = hashes
    
    bins = [{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}]

    for sample_hash in unique_hashes:
        
        sql = ("SELECT song_id, offset from fingerprints where hash = %s")
        val = (sample_hash[1],)
        mycursor.execute(sql,val)
        id_and_offset = mycursor.fetchall()
        sample_offset = sample_hash[2] 
        for j in id_and_offset:
            
            song_id = j[0]
            database_offset = j[1]
            difference = database_offset - sample_offset
            bins[song_id][difference] = bins[song_id].get(difference,0) + 1 
    
    score = {}
    for i in range(len(bins)):
        if bins[i] != {}:
            score[i] = bins[i].get( max(bins[i], key = bins[i].get) )
    try:
        Best_match = max(score,key = score.get)
    except:
        print("Error")
        return 
    #Get the name of the song with the song_id equal to best_match
    sql = ("SELECT song_name from songs where song_id = %s")
    val = (Best_match,)
    mycursor.execute(sql,val)
    name_best_match = mycursor.fetchone()
    
    print("The best match song is ", name_best_match[0]," with a score of ",score[Best_match])
    
    return


In [ ]:
print("What do you wish to do:")
print("    1. Add songs to the data base")
print("    2. Recognize a song")
choice = input("Enter your choice number: ")
if (choice == '1'):
    add_songs_to_database()
elif (choice =='2'):
    recognize()
else:
    print("PLease enter a valid choice number")